In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier

from matplotlib import pyplot as plt


/Users/bradeneberhard/miniforge3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
DATA = pd.read_csv('~/Downloads/all_games_test.csv')

/var/folders/r4/cqt7s71n4k18qp94swd6_njr0000gn/T/ipykernel_74406/426526817.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  DATA = pd.read_csv('~/Downloads/all_games_test.csv')


In [33]:
this_year = ['2023' in x for x in DATA.gameID]
THIS_YEAR = DATA[this_year & DATA.turnover == 1]

In [45]:
this_year = ['2023' in x for x in DATA.gameID]
THIS_YEAR = DATA[this_year & DATA.turnover == 1]

rows = []
for team in THIS_YEAR.home_teamID.unique():
    row = {}
    TEST = THIS_YEAR[(THIS_YEAR.home_teamID == team) | (THIS_YEAR.away_teamID == team)]
    row['team'] = team
    row['total_turns'] = len(TEST)
    row['d_line_turns'] = len(TEST[TEST.start_on_offense == False])
    row['o_line_turns'] = len(TEST[TEST.start_on_offense == True])
    rows.append(row)

turnover_df = pd.DataFrame(rows)
turnover_df['o_line_turnover_%'] = turnover_df.o_line_turns / turnover_df.total_turns
turnover_df['d_line_turnover_%'] = turnover_df.d_line_turns / turnover_df.total_turns

this_year = ['2023' in x for x in DATA.gameID]
THIS_YEAR = DATA

rows = []
for team in THIS_YEAR.home_teamID.unique():
    row = {}
    TEST = THIS_YEAR[(THIS_YEAR.home_teamID == team) | (THIS_YEAR.away_teamID == team)]
    row['team'] = team
    row['total_throws'] = len(TEST)
    row['d_line_throws'] = len(TEST[TEST.start_on_offense == False])
    row['o_line_throws'] = len(TEST[TEST.start_on_offense == True])
    rows.append(row)

throws_df = pd.DataFrame(rows)
throws_df['o_line_throw_%'] = throws_df.o_line_throws / throws_df.total_throws
throws_df['d_line_throw_%'] = throws_df.d_line_throws / throws_df.total_throws


In [47]:
final_df = pd.merge(turnover_df, throws_df, on='team')
final_df['percent_turns_over_throws_o'] = final_df['o_line_turnover_%'] / final_df['o_line_throw_%']
final_df['percent_turns_over_throws_d'] = final_df['d_line_turnover_%'] / final_df['d_line_throw_%']

In [50]:
final_df.round(2).sort_values('percent_turns_over_turnover_o')

,team,total_turns,d_line_turns,o_line_turns,o_line_turnover_%,d_line_turnover_%,total_throws,d_line_throws,o_line_throws,o_line_throw_%,d_line_throw_%,percent_turns_over_throws_o,percent_turns_over_throws_d
7,rush,305,110,193,0.63,0.36,12733,3119,9608,0.75,0.24,0.84,1.47
19,empire,283,97,186,0.66,0.34,18546,4174,14366,0.77,0.23,0.85,1.52
20,glory,275,94,181,0.66,0.34,14681,3749,10932,0.74,0.26,0.88,1.34
14,flyers,313,100,213,0.68,0.32,19192,4302,14890,0.78,0.22,0.88,1.43
18,thunderbirds,268,85,183,0.68,0.32,14597,3364,11233,0.77,0.23,0.89,1.38
13,havoc,399,140,259,0.65,0.35,3920,1061,2859,0.73,0.27,0.89,1.30
1,spiders,308,102,206,0.67,0.33,15962,4144,11812,0.74,0.26,0.90,1.28
5,royal,303,103,200,0.66,0.34,12640,3395,9245,0.73,0.27,0.90,1.27
17,hustle,250,78,172,0.69,0.31,16347,3876,12459,0.76,0.24,0.90,1.32
22,growlers,272,89,183,0.67,0.33,16745,4216,12529,0.75,0.25,0.90,1.30


In [36]:
df.sort_values('d_line_%')

,team,total_turns,d_line_turns,o_line_turns,o_line_%,d_line_%
9,alleycats,207,57,150,0.724638,0.275362
8,phoenix,306,88,216,0.705882,0.287582
21,summit,268,78,190,0.708955,0.291045
11,union,217,64,153,0.705069,0.294931
16,nitro,258,77,181,0.701550,0.298450
12,legion,268,80,188,0.701493,0.298507
23,radicals,211,63,148,0.701422,0.298578
15,cascades,318,95,223,0.701258,0.298742
6,breeze,207,62,145,0.700483,0.299517
4,aviators,307,94,213,0.693811,0.306189


In [6]:
DATA[DATA.turnover == 1]

,thrower,thrower_x,thrower_y,receiver,receiver_x,receiver_y,turnover,start_on_offense,point_start_time,current_line,...,game_quarter,quarter_point,point_timeouts,coming_from_recording_team_penalty,coming_from_opposing_team_penalty,home_team_score,away_team_score,gameID,home_teamID,away_teamID
13,cweinberg,-13.05,76.74,NaN,-17.35,101.57,1,True,0.0,"['jklane', 'nvogt', 'cjurek', 'rlinehan', 'aro...",...,1,0,0,0,0,0,0,2021-07-11-MIN-CHI,union,windchill
14,ddremann,20.32,18.88,NaN,20.32,18.88,1,False,0.0,"['valanguil', 'ngoff', 'jvallee', 'ddremann', ...",...,1,0,0,0,0,0,0,2021-07-11-MIN-CHI,union,windchill
52,knorth,-6.21,61.02,NaN,-14.40,108.34,1,True,255.0,"['pshriwise', 'parters', 'nlake', 'knorth', 'r...",...,1,0,0,0,0,1,2,2021-07-11-MIN-CHI,union,windchill
55,bvohnoutk,8.01,32.62,NaN,16.97,58.98,1,False,255.0,"['kjohnson1', 'jtaylor', 'bhidaka', 'bmatis', ...",...,1,0,0,0,0,1,2,2021-07-11-MIN-CHI,union,windchill
88,parters,5.25,56.65,NaN,5.25,56.65,1,True,462.0,"['eartemaki', 'pshriwise', 'parters', 'nlake',...",...,1,0,0,0,0,3,4,2021-07-11-MIN-CHI,union,windchill
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187316,powens,18.57,71.57,NaN,0.36,92.40,1,True,272.0,"['alillis', 'mcampanel', 'bdunn', 'aprato', 'e...",...,4,0,0,0,0,22,17,2022-07-30-OTT-PHI,phoenix,outlaws
187323,wdickson,12.17,50.69,NaN,22.09,59.84,1,False,272.0,"['aarsenaul1', 'ccampbell', 'ckennedy', 'gelli...",...,4,0,0,0,0,22,17,2022-07-30-OTT-PHI,phoenix,outlaws
187338,iwallace,-13.11,40.00,mayad,8.75,95.89,1,True,575.0,"['jhill', 'iwallace', 'mayad', 'ckennedy', 'ge...",...,4,0,0,0,0,24,18,2022-07-30-OTT-PHI,phoenix,outlaws
187339,enardelli,-7.80,37.35,NaN,-9.04,39.24,1,False,575.0,"['alillis', 'mcampanel', 'aprato', 'ewitmer', ...",...,4,0,0,0,0,24,18,2022-07-30-OTT-PHI,phoenix,outlaws
